In [1]:
%load_ext watermark
%watermark -a Chan -d -v -p numpy,pandas,konlpy,tensorflow

Chan 2019-08-09 

CPython 3.7.3
IPython 7.6.1

numpy 1.16.4
pandas 0.24.2
konlpy 0.5.1
tensorflow 1.13.1


In [2]:
import numpy as np
import pandas as pd
from konlpy.tag import Okt
import tensorflow as tf
from tensorflow.python.keras.preprocessing import sequence
from tensorflow import keras
from tqdm import tqdm
from collections import Counter
import pickle

In [3]:
# 각 동영상별로 count해야한다. 일단 dump파일로 짜놓기

In [28]:
dump_data = pd.read_csv("../../data/sample (7).csv", engine='python',names=['chat', 'label'])

In [29]:
dump_data.head()

,chat,label
0,"님, 50000원 후원 그라샤! - 브베야 오늘 이것만 주고 간다. 그리고 춤좀 그...",0
1,어켜졌당,0
2,브베형,0
3,아니,0
4,싹둑이 채팅창 관리 중입니다,0


In [30]:
len(dump_data)

500

In [31]:
morp = Okt()
morped = [morp.morphs(_, norm=True, stem=True) for _ in tqdm(dump_data.chat)]


100%|███████████████████████████████████████████████████████████████████████████████| 500/500 [00:02<00:00, 193.76it/s]

In [32]:
morped

[['님',
  ',',
  '50000원',
  '후원',
  '그라샤',
  '!',
  '-',
  '브베',
  '야',
  '오늘',
  '이',
  '것',
  '만',
  '주다',
  '간다',
  '.',
  '그리고',
  '춤좀',
  '그만',
  '추다',
  '땅',
  '꺼지다'],
 ['어', '켜지다'],
 ['브베', '형'],
 ['아니다'],
 ['싹둑', '이', '채팅', '창', '관리', '중', '이다'],
 ['왜', '또', '늦었샤', 'ㅡㅡ'],
 ['?!????', '???'],
 ['왜캐늦게왓샤'],
 ['싹둑', '이', '채팅', '창', '관리', '중', '이다'],
 ['야',
  '브베',
  '야',
  '너',
  '난리',
  '나다',
  '어제',
  '일',
  '때문',
  '에',
  '기자',
  '들',
  '이',
  '너',
  '찾다',
  '~'],
 ['브베', '미애', '장례식', '치르다', '오다'],
 ['케', '_', '인', '부터', '해명', '하고', '하자'],
 ['타', '스트리머', '비다', '신', '낫', '네'],
 ['나', '팔로우', '햇샤'],
 ['SabaPing'],
 ['미치다', '새', 'ㅡ', '끼'],
 ['싹둑', '이', '채팅', '창', '관리', '중', '이다'],
 ['지금', '이', '노래', '부르다', '상황', '이냐', '?'],
 ['냔냔냔'],
 ['쁘뼤', '야', '땨댱해', '창슥', '아'],
 ['오늘', '기분', '도', '안좋다', '브베', '방송', '이나', '보고', '풀다'],
 ['시작', '부터', '도우넛', '미치다'],
 ['어제', '아무렇다', '이유', '없이', '너', '한테', '욕먹다', '거', '기분', '이', '나쁘다'],
 ['ㄲ'],
 ['얘', '들', '아', '오늘', '도', '잘', '부다', '탁하다', '.'],
 ['싹둑

In [33]:
# load vocab
with open('./vocab/vocab_index.pickle', 'rb') as f:
    vocab_index = pickle.load(f)
with open('./vocab/vocab_inverted_index.pickle', 'rb') as f:
    vocab_inverted_index = pickle.load(f)

In [34]:
def index_to_text(indexes):
    return ' '.join([word_inverted_index[i] for i in indexes])

def text_to_index(tokens):
    oov_id = 1
    indexes = []
    for tok in tokens:
        if tok in vocab_index:
            indexes.append(vocab_index[tok])
        else:
            indexes.append(oov_id)

    return indexes

In [35]:
pad_id = 0

x_variable = [text_to_index(_) for _ in morped]

sentence_size = 20
x_padded = sequence.pad_sequences(x_variable,
                                 maxlen=sentence_size,
                                 truncating='post',
                                 padding='post',
                                 value=pad_id)

# Load Model 

In [36]:
new_model = keras.models.load_model('../model/cnn_oversample.h5')
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, None, 10)          80000     
_________________________________________________________________
conv1d_21 (Conv1D)           (None, None, 32)          992       
_________________________________________________________________
dropout_17 (Dropout)         (None, None, 32)          0         
_________________________________________________________________
conv1d_22 (Conv1D)           (None, None, 32)          3104      
_________________________________________________________________
global_max_pooling1d_10 (Glo (None, 32)                0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 66        
Total para

In [37]:
dump_data['predict'] = new_model.predict_classes(x_padded)

In [38]:
new_model.evaluate(x_padded, pd.get_dummies(dump_data.label)) # loss, acc

500/500 [==============================] - ETA: 6s - loss: 0.0646 - recall_16: 1.000 - 0s 929us/sample - loss: 0.6484 - recall_16: 0.8860


[0.6484258425235748, 0.886]

#### 전체 채팅 중 유해채팅 비율

In [46]:
dump_data.predict.sum() / len(dump_data)

0.084

> 5%가 적당하다

In [40]:
dump_data[dump_data.predict!=dump_data.label]

,chat,label,predict
37,나이쳐먹고 왜그렇게 노냐 브베야,1,0
47,브베 오만원 받음/,0,1
61,ㅅ ㅂ 맨날늦어,1,0
77,안녕하세요. 뽀뽀해주세요.,0,1
85,브베 병띤 샊이,1,0
87,옷에서 냄새안나냐 씨1발련아,1,0
89,이새기 후원으로 욕하지마라 속으로 좋아하면서 그돈으로 안뫄방간다,1,0
92,바빠섴ㅋㅋ,0,1
105,밥쳐먹는 시간을 줄이면 충분히 옷 갈아입겠다,1,0
106,"니가 빌게이츠냐 시,바라",1,0
